# Hungarian Method for Solving the Assignment Problem


This notebook implements the Hungarian Algorithm for solving the assignment problem (minimization case).
The approach follows the classic steps described in operations research and outlined in the provided documentation:
1. Create a square cost matrix.
2. Subtract row and column minima.
3. Make initial assignments.
4. Cover all zeros with a minimum number of lines.
5. Adjust the matrix if an optimal assignment is not yet possible.
6. Repeat until the optimal solution is obtained.


## Step 1: Import Required Library

In [10]:
import numpy as np

## Step 2: Define the Hungarian Algorithm Function

In [11]:

import numpy as np

def hungarian_algorithm(cost_matrix):
    cost_matrix = np.array(cost_matrix)
    n, m = cost_matrix.shape

    if n != m:
        size = max(n, m)
        new_matrix = np.zeros((size, size))
        new_matrix[:n, :m] = cost_matrix
        cost_matrix = new_matrix
        n = m = size

    for i in range(n):
        cost_matrix[i] -= cost_matrix[i].min()

    for j in range(m):
        cost_matrix[:, j] -= cost_matrix[:, j].min()

    def find_assignments(matrix):
        n = len(matrix)
        m = len(matrix[0])
        assigned = [-1] * m  # assigned[j] = i means row i assigned to column j

        def bpm(u, seen, match_r):
            for v in range(m):
                if matrix[u][v] == 0 and not seen[v]:
                    seen[v] = True
                    if match_r[v] == -1 or bpm(match_r[v], seen, match_r):
                        match_r[v] = u
                        return True
            return False

        match_r = [-1] * m
        for u in range(n):
            seen = [False] * m
            bpm(u, seen, match_r)

        result_matrix = np.zeros((n, m), dtype=int)
        for j in range(m):
            if match_r[j] != -1:
                result_matrix[match_r[j], j] = 1
        return result_matrix  

    def cover_zeros(matrix, assigned):
        covered_rows = np.zeros(n, dtype=bool)
        covered_cols = np.zeros(m, dtype=bool)
        marked_rows = np.zeros(n, dtype=bool)

        for i in range(n):
            if not any(assigned[i]):
                marked_rows[i] = True

        changed = True
        while changed:
            changed = False
            for i in range(n):
                if marked_rows[i]:
                    for j in range(m):
                        if matrix[i, j] == 0 and not covered_cols[j]:
                            covered_cols[j] = True
                            changed = True
            for j in range(m):
                if covered_cols[j]:
                    for i in range(n):
                        if assigned[i, j] and not marked_rows[i]:
                            marked_rows[i] = True
                            changed = True

        for i in range(n):
            if not marked_rows[i]:
                covered_rows[i] = True

        return covered_rows, covered_cols

    while True:
        assigned = find_assignments(cost_matrix)
        total_assignments = np.sum(assigned)

        if total_assignments == n:
            break

        covered_rows, covered_cols = cover_zeros(cost_matrix, assigned)

        uncovered_values = [
            cost_matrix[i, j]
            for i in range(n)
            for j in range(m)
            if not covered_rows[i] and not covered_cols[j]
        ]
        if not uncovered_values:
            break
        min_uncovered = min(uncovered_values)

        for i in range(n):
            for j in range(m):
                if not covered_rows[i] and not covered_cols[j]:
                    cost_matrix[i, j] -= min_uncovered
                elif covered_rows[i] and covered_cols[j]:
                    cost_matrix[i, j] += min_uncovered

    result = []
    for i in range(n):
        for j in range(m):
            if assigned[i, j] == 1:
                result.append((i, j))

    return result, cost_matrix

## Step 3: Provide a Cost Matrix and Solve

In [13]:

# cost_matrix = [
#     [90, 75, 75, 80],
#     [35, 85, 55, 65],
#     [125, 95, 90, 105],
#     [45, 110, 95, 115]
# ]
# cost_matrix = [
#     [120, 100, 80],
#     [80, 90, 110],
#     [110, 140, 120]
# ]

cost_matrix = [
    [10,  5, 13, 15, 16],
    [ 3,  9, 18, 13,  6],
    [10,  7,  2,  2,  2],
    [ 7, 11,  9,  7, 12],
    [ 7,  9, 10,  4, 12]
]

assignments, final_matrix = hungarian_algorithm(cost_matrix)

print("Optimal Assignments (row, column):")
for row, col in assignments:
    print(f"Row {row} -> Column {col}")

total_cost = sum(cost_matrix[row][col] for row, col in assignments)
print("Total Cost:", total_cost)


Optimal Assignments (row, column):
Row 0 -> Column 1
Row 1 -> Column 0
Row 2 -> Column 4
Row 3 -> Column 2
Row 4 -> Column 3
Total Cost: 23
